In [3]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt

import seaborn as sns



In [4]:
# Utility functions from Tutorial
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

In [5]:
SEED = 1
random.seed(SEED)
np.random.seed(SEED)

In [6]:
Kenya = pd.read_csv("GeoAI/Agricultural Plastic/Data/Kenya_training.csv")
Spain =  pd.read_csv("GeoAI/Agricultural Plastic/Data/Spain_training.csv")
VNM =  pd.read_csv("GeoAI/Agricultural Plastic/Data/VNM_training.csv")
VNM.rename(columns={'Lat': 'lat', 'Lon': 'lon'}, inplace=True)
df = pd.concat([Kenya, Spain, VNM], axis=0)

In [7]:
y_ = df['TARGET']-1
X_ = df[['lat', 'lon', 'blue_p50', 'green_p50', 'nir_p50', 'nira_p50', 're1_p50', 're2_p50', 
         're3_p50', 'red_p50', 'swir1_p50', 'swir2_p50', 'VV_p50', 'VH_p50']]

In [8]:
X_train, X_eval, y_train, y_eval = train_test_split(X_, y_, test_size=0.1, random_state=1)

In [9]:
from sklearn.ensemble import RandomForestClassifier

RFR = RandomForestClassifier()
RFR.fit(X_train, y_train)
RFR.score(X_eval, y_eval)

0.9611307420494699

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

GBR = GradientBoostingClassifier()
GBR.fit(X_train, y_train)
GBR.score(X_eval, y_eval)

0.9646643109540636

In [12]:
pip list --outdated

Package           Version   Latest Type
----------------- --------- ------ -----
attrs             23.2.0    24.1.0 wheel
debugpy           1.8.2     1.8.3  sdist
exceptiongroup    1.2.1     1.2.2  wheel
jupyter_server    2.14.1    2.14.2 wheel
jupyterlab        4.2.3     4.2.4  wheel
jupyterlab_server 2.27.2    2.27.3 wheel
pip               24.1.2    24.2   wheel
plotly            5.22.0    5.23.0 wheel
pure-eval         0.2.2     0.2.3  wheel
pyzmq             26.0.3    26.1.0 wheel
rpds-py           0.19.0    0.19.1 wheel
setuptools        68.2.2    72.1.0 wheel
sympy             1.13.0    1.13.1 wheel
tenacity          8.5.0     9.0.0  wheel
torch             2.3.1+cpu 2.4.0  wheel
urllib3           2.0.7     2.2.2  wheel

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
from catboost import CatBoostClassifier

CAT = CatBoostClassifier()
CAT.fit(X_train, y_train)
print(CAT.score(X_eval, y_eval))

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [14]:
from lightgbm import LGBMClassifier

LGB = LGBMClassifier(force_col_wise=True)
LGB.fit(X_train, y_train)
print(LGB.score(X_eval, y_eval))

[LightGBM] [Info] Number of positive: 1324, number of negative: 1218
[LightGBM] [Info] Total Bins 3570
[LightGBM] [Info] Number of data points in the train set: 2542, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.520850 -> initscore=0.083447
[LightGBM] [Info] Start training from score 0.083447
0.9752650176678446


In [15]:
from xgboost import XGBClassifier
XGB = XGBClassifier()
XGB.fit(X_train, y_train, eval_set=[(X_eval, y_eval)], verbose=False)
print(XGB.score(X_eval, y_eval))

0.9752650176678446


### Test with Eval

In [16]:
from sklearn.metrics import accuracy_score

rfr  = RFR.predict(X_eval)
gbr  = GBR.predict(X_eval)
#cat  = GBR.predict(X_eval)
lgb = LGB.predict(X_eval)
xgb = XGB.predict(X_eval)
pred = np.round((rfr+gbr+cat+lgb+xgb)/5,0).astype(int)
print(accuracy_score(y_eval, pred))

rfr  = RFR.predict(X_eval)*0.96
gbr  = GBR.predict(X_eval)*0.96
#cat  = GBR.predict(X_eval)*0.97
lgb = LGB.predict(X_eval)*0.98
xgb = XGB.predict(X_eval)*0.98
pred = np.round((rfr+gbr+cat+lgb+xgb)/(0.96+0.96+0.97+0.98+0.98)).astype(int)
print(accuracy_score(y_eval, pred))


rfr  = RFR.predict_proba(X_eval)*0.96
gbr  = GBR.predict_proba(X_eval)*0.96
#cat  = GBR.predict_proba(X_eval)*0.97
lgb = LGB.predict_proba(X_eval)*0.98
xgb = XGB.predict_proba(X_eval)*0.98
pred = (rfr+gbr+cat+lgb+xgb)/(0.96+0.96+0.97+0.98+0.98)
pred = np.argmax(pred, axis=1)
print(accuracy_score(y_eval, pred))


NameError: name 'cat' is not defined

## Submission

In [17]:
Kenya = pd.read_csv("GeoAI/Agricultural Plastic/Data/Kenya_testing.csv")
Spain =  pd.read_csv("GeoAI/Agricultural Plastic/Data/Spain_validation.csv")
VNM =  pd.read_csv("GeoAI/Agricultural Plastic/Data/VNM_testing.csv")
VNM.rename(columns={'Lon': 'lon', 'Lat': 'lat'}, inplace=True)

submission = pd.DataFrame()
country = ['Kenya', 'Spain', 'VNM']
files = [Kenya, Spain, VNM]

In [20]:
Kenya.to_csv("GeoAI/Agricultural Plastic/Data/teste.csv")

In [18]:
for i in range(3):
    tmp = pd.DataFrame()
    # Create ID for submission file
    ID = [country[i] + '_' + str(x) for x in files[i]['ID']]
    # Predict
    X_ = files[i][['lat', 'lon', 'blue_p50', 'green_p50', 'nir_p50', 'nira_p50', 're1_p50', 're2_p50', 
         're3_p50', 'red_p50', 'swir1_p50', 'swir2_p50', 'VV_p50', 'VH_p50']]
    rfr  = RFR.predict_proba(X_)*0.96
    gbr  = GBR.predict_proba(X_)*0.96
    cat  = GBR.predict_proba(X_)*0.97
    lgb = LGB.predict_proba(X_)*0.98
    xgb = XGB.predict_proba(X_)*0.98

    pred = (rfr+gbr+cat+lgb+xgb)/(0.96+0.96+0.97+0.98+0.98)
    pred = np.argmax(pred, axis=1)
    TARGET = pred+1
    
    tmp["ID"] = ID
    tmp["TARGET"] = TARGET
    submission = pd.concat([submission, tmp], axis=0)


## Verificacao

In [23]:
import os
import pandas as pd

output_dir = 'GeoAI/Agricultural Plastic/Data/output'
os.makedirs(output_dir, exist_ok=True)  # Create output directory if it doesn't exist
output_file = os.path.join(output_dir, 'sub_0208_1150.csv')

# Save to CSV
submission.to_csv(output_file, index=False)
print(f"CSV file saved to: {output_file}")

CSV file saved to: GeoAI/Agricultural Plastic/Data/output/sub_0208_1150.csv


In [22]:
#submission.to_csv('GeoAI/Agricultural Plastic/Data/sub_0208_1150.csv',index=False)
submission.to_csv('GeoAI/Agricultural Plastic/Data/output/sub_0208_1150.csv', index=False)

OSError: Cannot save file into a non-existent directory: 'GeoAI/Agricultural Plastic/Data/output'

In [59]:
consolidado_0 = pd.read_csv('./Data/submission_0208_1150.csv')
consolidado_2 = pd.read_csv('GeoAI/Agricultural Plastic/Data/submission_2.csv')

c = pd.DataFrame()
c['s0'] = consolidado_0['TARGET']
c['s2'] = consolidado_2['TARGET']

c['diferente'] = (c['s0'] != c['s2'])

# Contando quantas vezes os valores são diferentes
num_dif = c['diferente'].sum()
num_dif

num_dif/c.shape[0]

0.007779349363507779

In [60]:
submission.shape

(2828, 2)